# API Binance


https://api.binance.com/api/v1/ticker/24hr

## Function connect_to_redshift

In [1]:
import psycopg2
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logging.info("Libraries calls ok")

def connect_to_redshift(host, dbname, user, password, port=5439):
    """
    Establishes a connection to a Redshift database.

    Parameters:
    - host (str): The hostname or IP address of the Redshift cluster.
    - dbname (str): The name of the Redshift database to connect to.
    - user (str): The username for authenticating with the Redshift database.
    - password (str): The password for authenticating with the Redshift database.
    - port (int, optional): The port number on which the Redshift cluster is listening. Default is 5439.

    Returns:
    - conn: A psycopg2 connection object representing the connection to the Redshift database.

    Raises:
    - Exception: If an error occurs during the connection attempt.
    """
    try:
        conn = psycopg2.connect(
            dbname=dbname,
            user=user,
            password=password,
            host=host,
            port=port
        )
        logging.info("Connection established")
        return conn
    except Exception as error:
        logging.info(f"An error occurred: {error}")


2024-04-01 12:56:00,378 - INFO - Libraries calls ok


## Function execute_query

In [2]:
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logging.info("Libraries calls ok")

def execute_query(connection, query):
    """
    Executes a SQL query on the provided database connection.

    Parameters:
    - connection: psycopg2 connection object representing the connection to the database.
    - query (str): The SQL query to be executed.

    Returns:
    - None

    Raises:
    - Exception: If an error occurs during the query execution.
    """
    try:
        cursor = connection.cursor()
        cursor.execute(query)
        connection.commit()
        logging.info("Query executed successfully")
    except Exception as error:
        logging.info(f"An error occurred: {error}")


2024-04-01 12:56:00,532 - INFO - Libraries calls ok


### Truncate table mateobelossi_coderhouse.binance_coins if exists.

In [3]:
dbname = "data-engineer-database"
port = 5439
host = "data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com"
table_name = "binance_coins"
schema_name = "mateobelossi_coderhouse"
user = "mateobelossi_coderhouse"
with open("/home/mateo/Desktop/Curso DE/✍️Consignas de Pre.entregas y Proyecto Final-20240331T125607Z-001/1er_pre_entrega/password_redshift.txt ",'r') as f:
    password= f.read()

logging.info("Connecting to redshift...")
conn = connect_to_redshift(host, dbname, user, password, port)
cur = conn.cursor()

logging.info("Checking if table exists...")
cur.execute(f"SELECT EXISTS (SELECT 1 FROM information_schema.tables WHERE table_schema = '{schema_name}' AND table_name = '{table_name}')")
table_exists = cur.fetchone()[0]
logging.info(f"Table exists ? : {table_exists}")

if table_exists:
    logging.info(f"Truncating table {schema_name}.{table_name}")
    query = f"truncate table {schema_name}.{table_name}"
    execute_query(conn, query)
else:
    logging.info(f"table {schema_name}.{table_name} do not exist.")

cur.close()
conn.close()

2024-04-01 12:56:00,670 - INFO - Connecting to redshift...


2024-04-01 12:56:02,210 - INFO - Connection established
2024-04-01 12:56:02,214 - INFO - Checking if table exists...
2024-04-01 12:56:02,634 - INFO - Table exists ? : True
2024-04-01 12:56:02,636 - INFO - Truncating table mateobelossi_coderhouse.binance_coins
2024-04-01 12:56:03,084 - INFO - Query executed successfully


### Create table mateobelossi_coderhouse.binance_coins if not exits.

In [4]:
dbname = "data-engineer-database"
user = "mateobelossi_coderhouse"
with open("/home/mateo/Desktop/Curso DE/✍️Consignas de Pre.entregas y Proyecto Final-20240331T125607Z-001/1er_pre_entrega/password_redshift.txt ",'r') as f:
    password= f.read()
port = 5439
host = "data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com"
table_name = "binance_coins"
schema_name = "mateobelossi_coderhouse"

CREATE_TABLE =f"""
CREATE TABLE {user}.{table_name} (
    symbol VARCHAR(256),
    priceChange FLOAT,
    priceChangePercent FLOAT,
    weightedAvgPrice FLOAT,
    prevClosePrice FLOAT,
    lastPrice FLOAT,
    lastQty FLOAT,
    bidPrice FLOAT,
    bidQty FLOAT,
    askPrice FLOAT,
    askQty FLOAT,
    openPrice FLOAT,
    highPrice FLOAT,
    lowPrice FLOAT,
    volume FLOAT,
    quoteVolume FLOAT,
    openTime BIGINT,
    closeTime BIGINT,
    firstId BIGINT,
    lastId BIGINT,
    count BIGINT,
    created_at TIMESTAMP
);
"""

logging.info("Connecting to redshift...")
conn = connect_to_redshift(host, dbname, user, password, port)
cur = conn.cursor()

logging.info("Checking if table exists...")
cur.execute(f"SELECT EXISTS (SELECT 1 FROM information_schema.tables WHERE table_schema = '{schema_name}' AND table_name = '{table_name}')")
table_exists = cur.fetchone()[0]
logging.info(f"Table exists ? : {table_exists}")

if not table_exists:
    logging.info(f"Creating table {schema_name}.{table_name}")
    cur.execute(
    CREATE_TABLE
    )
    conn.commit()
else:
    logging.info(f"table {schema_name}.{table_name} already exists.")

cur.close()
conn.close()

2024-04-01 12:56:03,122 - INFO - Connecting to redshift...


2024-04-01 12:56:04,344 - INFO - Connection established
2024-04-01 12:56:04,347 - INFO - Checking if table exists...
2024-04-01 12:56:04,752 - INFO - Table exists ? : True
2024-04-01 12:56:04,754 - INFO - table mateobelossi_coderhouse.binance_coins already exists.


#### 1) requests to api binance https://api.binance.com/api/v1/ticker/24hr
#### 2) Include a column named "created_at" in the obtained results, indicating the current time at the moment of making the requests.
#### 3) Save the results in a CSV file.
#### 4) Insert the results into Redshift.

In [5]:
import pandas as pd
from pandas.io.json import json_normalize
import requests, json
from datetime import datetime
import logging
from psycopg2.extras import execute_values

dbname = "data-engineer-database"
user = "mateobelossi_coderhouse"
with open("/home/mateo/Desktop/Curso DE/✍️Consignas de Pre.entregas y Proyecto Final-20240331T125607Z-001/1er_pre_entrega/password_redshift.txt ",'r') as f:
    password= f.read()
port = 5439
host = "data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com"
table_name = "binance_coins"
schema_name = "mateobelossi_coderhouse"

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logging.info("Libraries calls ok")

CUR_TIME = datetime.now().strftime("%Y%m%d_%H%M%S")
logging.info(f"Start Time : {CUR_TIME} ")


logging.info(f"Making the requests to API binance.")
r = requests.get('https://api.binance.com/api/v1/ticker/24hr')

if r.status_code == 200:
    logging.info(f"Connection ok ; Requests Status: {r.status_code}")
    result = r.json()
    result = json_normalize(result)

    logging.info("Adding a column named created_at with current time.")
    result['created_at'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    logging.info("Loading data from binance to a csv")
    name_csv = 'mercado_binance.csv'
    file_path = str("./binance/") + str(CUR_TIME) + '_' + str(name_csv)
    pd.DataFrame(result).to_csv(file_path, index=False)
    
    logging.info("Loading data from binance to redshift")
    with open(file_path, 'r') as f:
        conn = connect_to_redshift(host, dbname, user, password, port)
        data_to_insert = [tuple(row) for row in result.values]
        insert_query = f"""
            INSERT INTO {schema_name}.{table_name} (
                symbol, priceChange, priceChangePercent, weightedAvgPrice, prevClosePrice, lastPrice, lastQty,
                bidPrice, bidQty, askPrice, askQty, openPrice, highPrice, lowPrice, volume, quoteVolume,
                openTime, closeTime, firstId, lastId, count, created_at
            ) VALUES %s;
            """
        with conn.cursor() as cur:
            execute_values(
                cur,
                insert_query,
                data_to_insert,
                page_size=len(data_to_insert)
            )
            conn.commit()
            logging.info("Data inserted into Redshift successfully.")

else:
    logging.info(f"It was not able to connect to Binance ; Requests Status: {r.status_code}")

2024-04-01 12:56:05,308 - INFO - Libraries calls ok
2024-04-01 12:56:05,309 - INFO - Start Time : 20240401_125605 
2024-04-01 12:56:05,309 - INFO - Making the requests to API binance.
2024-04-01 12:56:06,033 - INFO - Connection ok ; Requests Status: 200
/tmp/ipykernel_51662/2226643561.py:30: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  result = json_normalize(result)
2024-04-01 12:56:06,093 - INFO - Adding a column named created_at with current time.
2024-04-01 12:56:06,097 - INFO - Loading data from binance to a csv
2024-04-01 12:56:06,135 - INFO - Loading data from binance to redshift
2024-04-01 12:56:07,335 - INFO - Connection established
2024-04-01 12:56:09,733 - INFO - Data inserted into Redshift successfully.


## Reading the data from Redshift.

In [6]:
import psycopg2
import pandas as pd

dbname = "data-engineer-database"
user = "mateobelossi_coderhouse"
with open("/home/mateo/Desktop/Curso DE/✍️Consignas de Pre.entregas y Proyecto Final-20240331T125607Z-001/1er_pre_entrega/password_redshift.txt ",'r') as f:
    password= f.read()
port = 5439
host = "data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com"
table_name = "binance_coins"
schema_name = "mateobelossi_coderhouse"

conn = psycopg2.connect(
    dbname=dbname,
    user=user,
    password=password,
    host=host,
    port=port
)

select_query = f"""
SELECT * FROM {schema_name}.{table_name}
"""

df = pd.read_sql(select_query, conn)

conn.close()


df.head()


/home/mateo/.local/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,symbol,pricechange,pricechangepercent,weightedavgprice,prevcloseprice,lastprice,lastqty,bidprice,bidqty,askprice,...,highprice,lowprice,volume,quotevolume,opentime,closetime,firstid,lastid,count,created_at
0,ETHBTC,-0.000900,-1.750,0.050967,0.051430,0.050540,1.2671,0.050530,46.7291,0.050540,...,0.051470,0.050430,29393.2139,1498.076447,1711900565817,1711986965817,438956384,439043594,87211,2024-04-01 12:56:06
1,LTCBTC,-0.000027,-1.844,0.001516,0.001464,0.001437,0.7970,0.001436,12.0820,0.001437,...,0.001590,0.001430,210313.0050,318.891932,1711900565794,1711986965794,96346451,96406320,59870,2024-04-01 12:56:06
2,BNBBTC,-0.000171,-1.998,0.008444,0.008559,0.008388,1.2170,0.008387,8.5250,0.008388,...,0.008600,0.008286,43155.0900,364.398836,1711900565787,1711986965787,238134341,238223501,89161,2024-04-01 12:56:06
3,NEOBTC,-0.000005,-2.145,0.000232,0.000228,0.000224,22.8900,0.000223,191.1800,0.000224,...,0.000237,0.000223,70986.9500,16.464070,1711900565500,1711986965500,46257841,46262865,5025,2024-04-01 12:56:06
4,QTUMETH,-0.000041,-3.104,0.001336,0.001318,0.001280,2.9000,0.001274,848.8000,0.001278,...,0.001355,0.001270,9859.9000,13.173310,1711900558724,1711986958724,5438812,5439019,208,2024-04-01 12:56:06
